In [1]:
# Load paths for using psana
%env SIT_ROOT=/reg/g/psdm/
%env SIT_DATA=/cds/group/psdm/data/
%env SIT_PSDM_DATA=/cds/data/psdm/

env: SIT_ROOT=/reg/g/psdm/
env: SIT_DATA=/cds/group/psdm/data/
env: SIT_PSDM_DATA=/cds/data/psdm/


In [2]:
import os
import torch
import random
import numpy as np
import h5py
import time

from peaknet.predictor import CheetahPeakFinder
from peaknet.datasets.utils import PsanaImg
from peaknet.datasets.transform import center_crop, coord_crop_to_img
from cupyx.scipy import ndimage
import cupy as cp

seed = 0

In [3]:
import matplotlib.pyplot as plt
import matplotlib.colors       as mcolors
import matplotlib.patches      as mpatches
import matplotlib.transforms   as mtransforms
%matplotlib inline

## Load psana

In [4]:
exp           = 'cxic0415'
run           = 101
img_load_mode = 'calib'
access_mode   = 'idx'
detector_name = 'CxiDs1.0:Cspad.0'
photon_energy = 12688.890590380644    # eV
encoder_value = -450.0034

psana_img = PsanaImg(exp, run, access_mode, detector_name)

## Load Model

In [5]:
timestamp = "2022_1101_2326_41"    # Manual

In [6]:
from peaknet.methods.unet import UNet
from peaknet.model        import ConfigPeakFinderModel, PeakFinderModel

In [7]:
base_channels = 8
pos_weight    = 1.0
focal_alpha   = 0.8
focal_gamma   = 2.0
method = UNet( in_channels = 1, out_channels = 1, base_channels = base_channels )
config_peakfinder = ConfigPeakFinderModel( method = method, 
                                           pos_weight = pos_weight, 
                                           focal_alpha = focal_alpha,
                                           focal_gamma = focal_gamma)
model = PeakFinderModel(config_peakfinder)
model.init_params()

In [8]:
model.init_params(from_timestamp = timestamp)   # Run this will load a trained model

In [9]:
# Load model to gpus if available...
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
model  = torch.nn.DataParallel(model.method).to(device)

## Load cheetah geometry

In [10]:
path_cheetah_geom = 'cheetah_geom.pickle'

## Enable Cheetah peak finder

In [11]:
pf = CheetahPeakFinder(model = model, path_cheetah_geom = path_cheetah_geom)

## Visual check

In [12]:
def load_img(event):
    multipanel_mask       = psana_img.create_bad_pixel_mask()
    multipanel_img        = psana_img.get(event, None, 'calib')
    multipanel_img_masked = multipanel_mask * multipanel_img
    
    return multipanel_img_masked

In [13]:
event = 3068
multipanel_img_masked = load_img(event)
img_stack = torch.tensor(multipanel_img_masked).type(dtype=torch.float)[:,None].to(device)

In [14]:
import time

time_start = time.time()
peak_list = pf.find_peak(img_stack, threshold_prob = 1 - 1e-4)
time_end = time.time()
time_delta = time_end - time_start
print(f"Time delta: {time_delta} second.")

Time delta: 0.39653468132019043 second.


In [ ]:
def convert_psana_to_cheetah(panel_list):
    # [!!!] Hard code
    dim0 = 8 * 185
    dim1 = 4 * 388

    # Convert calib image to cheetah image
    img = np.zeros((dim0, dim1))
    counter = 0
    for quad in range(4):
        for seg in range(8):
            img[seg * 185:(seg + 1) * 185, quad * 388:(quad + 1) * 388] = panel_list[counter, :, :]
            counter += 1

    return img

img_cheetah = convert_psana_to_cheetah(multipanel_img_masked)
size_y, size_x = img_cheetah.shape[-2:]

plt.figure(figsize = (60, 60))
data = img_cheetah
vmin = data.mean()
vmax = vmin + data.std() * 6
plt.imshow(img_cheetah, vmin = vmin, vmax = vmax)

# Place a box on a peak
offset = 3
b_offset = 10
for y, x in peak_list:
    x_bottom_left = x - offset
    y_bottom_left = y - offset

    rec_obj = mpatches.Rectangle((x_bottom_left, y_bottom_left), 
                                 2 * offset, 2 * offset, 
                                 linewidth = 1.0, 
                                 edgecolor = 'yellow', 
                                 facecolor='none')
    plt.gca().add_patch(rec_obj)

    y_bmin, x_bmin = 0, 0
    y_bmax, x_bmax = size_y, size_x
    plt.gca().set_xlim([x_bmin - b_offset, x_bmax + b_offset])
    plt.gca().set_ylim([y_bmin - b_offset, y_bmax + b_offset])

## Let's find it!!!

In [ ]:
min_num_peak = 20

event_start = 0
event_end   = 5000

multipanel_mask = psana_img.create_bad_pixel_mask()

event_filtered_list = []
for event in range(event_start, event_end):
    print(f"Processing {event:06d}... ", end = '')

    multipanel_img        = psana_img.get(event, None, 'calib')
    multipanel_img_masked = multipanel_mask * multipanel_img
    
    img_stack = torch.tensor(multipanel_img_masked).type(dtype=torch.float)[:,None].to(device)
    
    time_start = time.time()
    peak_list = pf.find_peak(img_stack, threshold_prob = 1 - 1e-4)
    time_end = time.time()
    time_delta = time_end - time_start
    print(f"Time delta: {time_delta:.6f} millisecond.")
    
    if len(peak_list) < min_num_peak: continue
    
    event_filtered_list.append([event, peak_list])

## Let's save it.

In [25]:
def convert_psana_to_cheetah(panel_list):
    # [!!!] Hard code
    dim0 = 8 * 185
    dim1 = 4 * 388

    # Convert calib image to cheetah image
    img = np.zeros((dim0, dim1))
    counter = 0
    for quad in range(4):
        for seg in range(8):
            img[seg * 185:(seg + 1) * 185, quad * 388:(quad + 1) * 388] = panel_list[counter, :, :]
            counter += 1

    return img

fl_cxi = 'peak.cxi'
max_num_peak = 2048

multipanel_mask = psana_img.create_bad_pixel_mask()

num_event = len(event_filtered_list)
with h5py.File(fl_cxi, 'w') as myHdf5:
    # [!!!] Hard code
    dim0 = 8 * 185
    dim1 = 4 * 388

    grpName     = "/entry_1/result_1"
    dset_nPeaks = "/nPeaks"
    dset_posX   = "/peakXPosRaw"
    dset_posY   = "/peakYPosRaw"
    dset_atot   = "/peakTotalIntensity"

    grp = myHdf5.create_group(grpName)
    myHdf5.create_dataset(grpName + dset_nPeaks, (num_event,             ), dtype='int')
    myHdf5.create_dataset(grpName + dset_posX  , (num_event, max_num_peak), dtype='float32', chunks=(1, max_num_peak))
    myHdf5.create_dataset(grpName + dset_posY  , (num_event, max_num_peak), dtype='float32', chunks=(1, max_num_peak))
    myHdf5.create_dataset(grpName + dset_atot  , (num_event, max_num_peak), dtype='float32', chunks=(1, max_num_peak))

    myHdf5.create_dataset("/LCLS/detector_1/EncoderValue", (1,), dtype=float)
    myHdf5.create_dataset("/LCLS/photon_energy_eV", (1,), dtype=float)
    dset = myHdf5.create_dataset("/entry_1/data_1/data", (num_event, dim0, dim1), dtype=np.float32) # change to float32
    ## dsetM = myHdf5.create_dataset("/entry_1/data_1/mask", (dim0, dim1), dtype='int')

    # # Save images...
    # for seqi, (event, _) in enumerate(event_filtered_list):
    #     multipanel_img_masked = load_img(event)
    #     dset[seqi] = convert_psana_to_cheetah(multipanel_img_masked)

    for seqi, (event, peak_per_event_list) in enumerate(event_filtered_list):
        # Save this event???
        nPeaks = len(peak_per_event_list)
        if nPeaks > max_num_peak: continue
        
        # Save images...
        multipanel_img        = psana_img.get(event, None, 'calib')
        multipanel_img_masked = multipanel_mask * multipanel_img
        dset[seqi] = convert_psana_to_cheetah(multipanel_img_masked)
        
        # Save peaks...
        for i, peak in enumerate(peak_per_event_list):
            cheetahRow, cheetahCol = peak
            myHdf5[grpName + dset_posX][seqi, i] = cheetahCol
            myHdf5[grpName + dset_posY][seqi, i] = cheetahRow
        myHdf5[grpName + dset_nPeaks][seqi] = nPeaks

    myHdf5["/LCLS/detector_1/EncoderValue"][0] = encoder_value  # mm
    myHdf5["/LCLS/photon_energy_eV"][0] = photon_energy

(1749, 1480, 1552)


In [26]:
fl_cxi = 'peak.cxi'
max_num_peak = 2048
drc = os.getcwd()
path_cxi = os.path.join(drc, fl_cxi)
basename = fl_cxi[:fl_cxi.rfind('.')]
fl_lst = f'{basename}.lst'
with open(fl_lst,'w') as fh:
    for i, (event, peak_per_event_list) in enumerate(event_filtered_list):
        # Index this event???
        nPeaks = len(peak_per_event_list)
        if nPeaks > max_num_peak: continue
        
        fh.write(f"{path_cxi} //{i}")
        fh.write("\n")